In [12]:
'''CODE HERE'''
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
# import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
# from sklearn.linear_model import LogisticRegression
# #A random forest is a meta estimator that fits a number of decision tree classifiers 
# #on various sub-samples of the dataset and use averaging to improve the predictive 
# #accuracy and control over-fitting.
# from sklearn.ensemble import RandomForestClassifier
# #a discriminative classifier formally defined by a separating hyperplane.
# from sklearn.svm import SVC
# #displayd data
# from IPython.display import display
# %matplotlib inline

df = pd.read_csv('Data/finalData.csv')
def cleanDates():
    for index,row in df.iterrows():
        date = row['Date'].split("/")
        date[2] ="20" +date[2]
        date = date[2] + "-" + date[1] + "-" + date[0]
        row['Date'] = date
        df.loc[index] = row
        
cleanDates()

In [13]:

##returns the matches between start date and end date// yyyy-mm-dd
##display(getMatchesBeforGivenDate("2008-08-16","2010-08-17"))
def getMatchesBeforGivenDate(data,start_date,end_date):
    mask = (data['Date'] <= end_date)
    return data[mask]

df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True)


In [14]:
startDate="2005-01-01"
points = {"WIN":3,"DRAW":2,"LOSE":0}
def getMatchesWithHomeTeam(data, teamName):
    return data.loc[data['HomeTeam'] == teamName]


def getMatchesWithAwayTeam(data, teamName):
    return data.loc[data['AwayTeam'] == teamName]


def getMatchesWithHomeTeamWin(data):
    return data.loc[data['FTR'] == "H"]


def getMatchesWithAwayTeamWin(data):
    return data.loc[data['FTR'] == "A"]


def getMatchesWithDraw(data):
    return data.loc[data['FTR'] == "D"]


def getHomeTeamWinPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    homeTeamWinRows = getMatchesWithHomeTeamWin(teamRows)
    return len(homeTeamWinRows) / len(teamRows)


def getHomeTeamDrawPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    homeTeamDrawRows = getMatchesWithDraw(teamRows)
    return len(homeTeamDrawRows) / len(teamRows)


def getAwayTeamWinPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    awayTeamWinRows = getMatchesWithAwayTeamWin(teamRows)
    return len(awayTeamWinRows) / len(teamRows)


def getAwayTeamDrawPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    awayTeamDrawRows = getMatchesWithDraw(teamRows)
    return len(awayTeamDrawRows) / len(teamRows)


def getAvgHomeTeamGoals(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalHomeGoals = teamRows['FTHG'].sum()
    # display(teamRows)
    return totalHomeGoals / len(teamRows)


def getAvgAwayTeamGoals(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalAwayGoals = teamRows['FTAG'].sum()
    # display(teamRows)
    return totalAwayGoals / len(teamRows)


# getAvgHomeTeamGoals("Arsenal","2008-09-30")

def getAvgHomeTeamShots(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalhomeshots = teamRows['HS'].sum()
    display(teamRows)
    return totalhomeshots / len(teamRows)


def getAverageHomeFeature(teamName, toDate, feature):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalHomeFeature = teamRows[feature].sum()
    # display(teamRows)
    return totalHomeFeature / len(teamRows)


def getAverageAwayFeature(teamName, toDate, feature):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalAwayFeature = teamRows[feature].sum()
    # display(teamRows)
    return totalAwayFeature / len(teamRows)


def getMatchesOfTeam(data,teamName):
    return pd.concat([getMatchesWithHomeTeam(data,teamName),getMatchesWithAwayTeam(data,teamName)])


def calculatePointsForTeam(data,teamName):
    sum = 0
    for index,row in data.iterrows():
        if ((row["FTR"] == "H" and row["HomeTeam"]==teamName) or (row["FTR"] == "A" and row["AwayTeam"]==teamName)):
            sum += points["WIN"]
        elif row["FTR"] == "D":
            sum += points["DRAW"]
    return sum
        
    
def getLastNGames(data,n):
    data = data.sort_values(['Date'], ascending=False)
    return data.head(n)
    
    
def getFormUntilNow(teamName,toDate,n):
    matchesInDateRange = getMatchesBeforGivenDate(df,startDate,toDate)
    matchesPlayedByTeam = getMatchesOfTeam(matchesInDateRange,teamName)
    if len(matchesPlayedByTeam) is 0:
        return 0
    lastNGames = getLastNGames(matchesPlayedByTeam,n)
    totalPoints = calculatePointsForTeam(lastNGames,teamName)
    return totalPoints

In [15]:
##Driver code
feature_vector = {}
teamIds ={}
teamId =1
for index, row in df.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    endTime = row['Date']
    label = row["FTR"]
    
    ##Assigning team ids
    if home_team not in teamIds:
        teamIds[home_team] = teamId
        teamId +=1
    if away_team not in teamIds:
        teamIds[away_team] = teamId
        teamId +=1
    ##Home team stats

    ##1.Home win %
    team1_home_win = getHomeTeamWinPercent(home_team, endTime)

    ##2.Away win %
    team1_away_win = getAwayTeamWinPercent(home_team, endTime)

    ##3.Home Draw
    team1_home_draw = getHomeTeamDrawPercent(home_team, endTime)
    ##4.Away Draw
    team1_away_draw = getAwayTeamDrawPercent(home_team, endTime)

    ##5. Average home goals
    team1_home_goals = getAvgHomeTeamGoals(home_team, endTime)
    ##6. Average away goals
    team1_away_goals = getAvgAwayTeamGoals(home_team, endTime)

    ##7.Average goals conceded playing home
    team1_home_goals_conceded = getAverageHomeFeature(home_team, endTime, "FTAG")
    ##8. Average goals conceded playing away
    team1_away_goals_conceded = getAverageAwayFeature(home_team, endTime, "FTHG")

    ##9. HS = Home Team Shots - take average
    team1_home_shots = getAverageHomeFeature(home_team, endTime, "HS")
    ##10. AS = Away Team Shots - take average
    team1_away_shots = getAverageAwayFeature(home_team, endTime, "AS")

    ##11. HST = Home Team Shots on Target -- take average
    team1_home_shots_target = getAverageHomeFeature(home_team, endTime, "HST")
    ##12. AST = Away Team Shots on Target - - take average
    team1_away_shots_target = getAverageHomeFeature(home_team, endTime, "AST")

    #     ##10. HHW = Home Team Hit Woodwork- take average
    #     team1_home_shots_wood =getAverageHomeFeature(home_team, endTime, "HHW")
    #     ##11. AHW = Away Team Hit Woodwork- take average
    #     team1_away_shots_wood =getAverageHomeFeature(home_team, endTime, "AHW")

    ##13. HC = Home Team Corners- take average
    team1_home_corners = getAverageHomeFeature(home_team, endTime, "HC")
    ##14. AC = Away Team Corners- take average
    team1_away_corners = getAverageHomeFeature(home_team, endTime, "AC")

    ##15. HF = Home Team Fouls Committed- take average
    team1_home_fouls = getAverageHomeFeature(home_team, endTime, "HF")
    ##16. AF = Away Team Fouls Committed- take average
    team1_away_fouls = getAverageAwayFeature(home_team, endTime, "AF")

    ##HFKC = Home Team Free Kicks Conceded- take average
    ##team1_home_fk =getAverageHomeFeature(home_team, endTime, "HFKC")
    ##17. AFKC = Away Team Free Kicks Conceded- take average
    ##team1_away_fk =getAverageHomeFeature(home_team, endTime, "AFKC")

    ## HO = Home Team Offsides- take average
    ##team1_home_off =getAverageHomeFeature(home_team, endTime, "HO")
    ## AO = Away Team Offsides- take average
    ##team1_away_off =getAverageAwayFeature(home_team, endTime, "AO")

    ## HY = Home Team Yellow Cards- take average
    ## AY = Away Team Yellow Cards- take average
    team1_home_yellow, team1_away_yellow = getAverageHomeFeature(home_team, endTime, "HY"), getAverageAwayFeature(home_team,
                                                                                                          endTime, "AY")
    ## HR = Home Team Red Cards- take average
    ##23. AR = Away Team Red Cards- take average
    team1_home_red, team1_away_red = getAverageHomeFeature(home_team, endTime, "HR"), getAverageAwayFeature(home_team, endTime,
                                                                                                    "AR")
    total_yellow = team1_home_yellow + team1_away_yellow
    total_Red = team1_home_red + team1_away_red

    ##17. HBP = Home Team Bookings Points (10 = yellow, 25 = red)- take average
    team1_home_foul_points = total_yellow * 10
    ##18. ABP = Away Team Bookings Points (10 = yellow, 25 = red)- take average
    team1_away_foul_points = total_Red * 25

    ##19. Last few games - form  in last 10 games.
    team1_form = getFormUntilNow(home_team, endTime, 10)
    ##################################################################
    ##################################################################
    ##Away team calculations
    ##1.Home win %
    team2_home_win = getHomeTeamWinPercent(away_team, endTime)

    ##2.Away win %
    team2_away_win = getAwayTeamWinPercent(away_team, endTime)

    ##3.Home Draw
    team2_home_draw = getHomeTeamDrawPercent(away_team, endTime)
    ##4.Away Draw
    team2_away_draw = getAwayTeamDrawPercent(away_team, endTime)

    ##5. Average home goals
    team2_home_goals = getAvgHomeTeamGoals(away_team, endTime)
    ##6. Average away goals
    team2_away_goals = getAvgAwayTeamGoals(away_team, endTime)

    ##7.Average goals conceded playing home
    team2_home_goals_conceded = getAverageHomeFeature(away_team, endTime, "FTAG")
    ##8. Average goals conceded playing away
    team2_away_goals_conceded = getAverageAwayFeature(away_team, endTime, "FTHG")

    ##9. HS = Home Team Shots - take average
    team2_home_shots = getAverageHomeFeature(away_team, endTime, "HS")
    ##10. AS = Away Team Shots - take average
    team2_away_shots = getAverageAwayFeature(away_team, endTime, "AS")

    ##11. HST = Home Team Shots on Target -- take average
    team2_home_shots_target = getAverageHomeFeature(away_team, endTime, "HST")
    ##12. AST = Away Team Shots on Target - - take average
    team2_away_shots_target = getAverageHomeFeature(away_team, endTime, "AST")

    #     ##10. HHW = Home Team Hit Woodwork- take average
    #     team2_home_shots_wood =getAverageHomeFeature(away_team, endTime, "HHW")
    #     ##11. AHW = Away Team Hit Woodwork- take average
    #     team2_away_shots_wood =getAverageHomeFeature(away_team, endTime, "AHW")

    ##13. HC = Home Team Corners- take average
    team2_home_corners = getAverageHomeFeature(away_team, endTime, "HC")
    ##14. AC = Away Team Corners- take average
    team2_away_corners = getAverageHomeFeature(away_team, endTime, "AC")

    ##15. HF = Home Team Fouls Committed- take average
    team2_home_fouls = getAverageHomeFeature(away_team, endTime, "HF")
    ##16. AF = Away Team Fouls Committed- take average
    team2_away_fouls = getAverageAwayFeature(away_team, endTime, "AF")

    ##HFKC = Home Team Free Kicks Conceded- take average
    ##team2_home_fk =getAverageHomeFeature(away_team, endTime, "HFKC")
    ##17. AFKC = Away Team Free Kicks Conceded- take average
    ##team2_away_fk =getAverageHomeFeature(away_team, endTime, "AFKC")

    ## HO = Home Team Offsides- take average
    ##team2_home_off =getAverageHomeFeature(away_team, endTime, "HO")
    ## AO = Away Team Offsides- take average
    ##team2_away_off =getAverageAwayFeature(away_team, endTime, "AO")

    ## HY = Home Team Yellow Cards- take average
    ## AY = Away Team Yellow Cards- take average
    team2_home_yellow, team2_away_yellow = getAverageHomeFeature(away_team, endTime, "HY"), getAverageAwayFeature(away_team,
                                                                                                                  endTime,
                                                                                                                  "AY")
    ## HR = Home Team Red Cards- take average
    ##23. AR = Away Team Red Cards- take average
    team2_home_red, team2_away_red = getAverageHomeFeature(away_team, endTime, "HR"), getAverageAwayFeature(away_team,
                                                                                                            endTime,
                                                                                                            "AR")
    total_yellow = team2_home_yellow + team2_away_yellow
    total_Red = team2_home_red + team2_away_red

    ##17. HBP = Home Team Bookings Points (10 = yellow, 25 = red)- take average
    team2_home_foul_points = total_yellow * 10
    ##18. ABP = Away Team Bookings Points (10 = yellow, 25 = red)- take average
    team2_away_foul_points = total_Red * 25

    ##19. Last few games - form  in last 10 games.
    team2_form = getFormUntilNow(away_team, endTime, 10)
    key = home_team+"_"+away_team+"_"+str(endTime).split(" ")[0]
    feature_vector[index] = [key,teamIds[home_team],teamIds[away_team],endTime,label,team1_home_win, team1_home_draw, team1_away_win, 
                             team1_away_draw,team1_home_goals, team1_away_goals,team1_home_goals_conceded, 
                             team1_away_goals_conceded,team1_home_shots, team1_away_shots, team1_home_shots_target,
                             team1_away_shots_target,team1_home_corners, team1_away_corners, team1_home_fouls,
                             team1_away_fouls, team1_home_foul_points, team1_away_foul_points, team1_form, 
                             team2_home_win, team2_home_draw, team2_away_win,
                             team2_away_draw, team2_home_goals, team2_away_goals,team2_home_goals_conceded, 
                             team2_away_goals_conceded,team2_home_shots, team2_away_shots, team2_home_shots_target, 
                             team2_away_shots_target,team2_home_corners, team2_away_corners, team2_home_fouls,
                             team2_away_fouls, team2_home_foul_points,team2_away_foul_points, team2_form,
                             row["B365H"],row["B365D"],row["B365A"],row["BWH"],row["BWD"],row["BWA"],row["GBH"],
                             row["GBD"],row["GBA"],row["IWH"],row["IWD"],row["IWA"],row["LBH"],row["LBD"],
                             row["LBA"],row["SBH"],row["SBD"],row["SBA"],row["WHH"],row["WHD"],row["WHA"],
                             row["SJH"],row["SJD"],row["SJA"],row["VCH"],row["VCD"],row["VCA"],row["BSH"],
                             row["BSD"],row["BSA"],row["Bb1X2"],row["BbMxH"],row["BbAvH"],row["BbMxD"],row["BbAvD"],
                             row["BbMxA"],row["BbAvA"],row["BbOU"],row["BbMx>2.5"],row["BbAv>2.5"],row["BbMx<2.5"],
                             row["BbAv<2.5"],row["BbAH"],row["BbAHh"],row["BbMxAHH"],row["BbAvAHH"],row["BbMxAHA"],
                             row["BbAvAHA"]]

In [16]:
###Creating a new csv out of the processed data!!
new_df = pd.DataFrame(list(feature_vector.values()))
new_df.columns = ["key","team1","team2","date","FTR","team1_home_win","team1_home_draw","team1_away_win",
                  "team1_away_draw","team1_home_goals", "team1_away_goals","team1_home_goals_conceded",
                  "team1_away_goals_conceded","team1_home_shots","team1_away_shots","team1_home_shots_target",
                  "team1_away_shots_target","team1_home_corners", "team1_away_corners","team1_home_fouls",
                  "team1_away_fouls","team1_home_foul_points", "team1_away_foul_points","team1_form",
                  "team2_home_win","team2_home_draw","team2_away_win",
                  "team2_away_draw","team2_home_goals","team2_away_goals","team2_home_goals_conceded",
                  "team2_away_goals_conceded","team2_home_shots","team2_away_shots","team2_home_shots_target",
                  "team2_away_shots_target","team2_home_corners","team2_away_corners","team2_home_fouls",
                  "team2_away_fouls","team2_home_foul_points","team2_away_foul_points","team2_form",
                  "Bsay n365H","B365D","B365A","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD",
                  "LBA","SBH","SBD","SBA","WHH","WHD","WHA","SJH","SJD","SJA","VCH","VCD","VCA","BSH","BSD",
                  "BSA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","BbOU","BbMx_2.5","BbAv_2.5",
                  "BbMx_2.5","BbAv_2.5","BbAH","BbAHh","BbMxAHH","BbAvAHH","BbMxAHA","BbAvAHA"]

# new_df[new_df==""] <- 0
new_df.to_csv("Data/final_processed_1.csv", sep=",")